In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from pynq import Overlay
overlay = Overlay('./bitstream/LUT20bits.bit')

In [3]:
overlay.ip_dict

{'LUT20bits_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf7b76d0>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'LUT20bits_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '4',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'LUT20bits_LUT20bits_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:LUT20bits:1.0'}}

In [4]:
from pynq import MMIO
ip = MMIO(0x43C00000, 0x10000)

In [5]:
ip.device

### 將 r_LUT  的location輸出值轉為2補數:

In [6]:
def twos_complement_32bit(x):
    """將 32-bit 補數整數轉為有號值"""
    if x >= (1 << 31):
        return x - (1 << 32)
    else:
        return x

#### Test:

In [8]:
location_1 = -14 
remainder_1 = 5858

ip.write_mm(0, location_1)
ip.write_mm(8, remainder_1)


remainder = ip.read_mm(4)
location = ip.read_mm(12)


print(remainder)
print(twos_complement_32bit(location))

4430
-30


### 開始則中查表驗證

## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 18613  (30bits), N的範圍: 0 ~ 1073741823 (30bits)

### Define function
* (Not including 2's complement range

In [9]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


In [18]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 20) - 1    ## 
data2 = (1 << 20) - 19
data3 = (1 << 20) - 15
AN = 6311
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)

inputs1的位元數: 33
inputs2的位元數: 33
inputs3的位元數: 33


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [19]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, 24, -24, 25, -25, 26, -26, 27, -27, 28, -28, 29, -29, 30, -30, 31, -31, 32, -32, 33, -33, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, 

### 產生SEC的模數:

In [20]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 6310, 2, 6309, 4, 6307, 8, 6303, 16, 6295, 32, 6279, 64, 6247, 128, 6183, 256, 6055, 512, 5799, 1024, 5287, 2048, 4263, 4096, 2215, 1881, 4430, 3762, 2549, 1213, 5098, 2426, 3885, 4852, 1459, 3393, 2918, 475, 5836, 950, 5361, 1900, 4411, 3800, 2511, 1289, 5022, 2578, 3733, 5156, 1155, 4001, 2310, 1691, 4620, 3382, 2929, 453, 5858, 906, 5405, 1812, 4499, 3624, 2687]
AWE1的模數:
 [1, 6310, 2, 6309, 4, 6307, 8, 6303, 16, 6295, 32, 6279, 64, 6247, 128, 6183, 256, 6055, 512, 5799, 1024, 5287, 2048, 4263, 4096, 2215, 1881, 4430, 3762, 2549, 1213, 5098, 2426, 3885, 4852, 1459, 3393, 2918, 475, 5836, 950, 5361, 1900, 4411, 3800, 2511, 1289, 5022, 2578, 3733, 5156, 1155, 4001, 2310, 1691, 4620, 3382, 2929, 453, 5858, 906, 5405, 1812, 4499, 3624, 2687]
AWE1的模數是否等於SEC模數: True
SEC的大小: 66


### 產生2-bit AWE (Arithmetic-Weight Error)

In [21]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 6617556828
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 6617556824
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 6617556826
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 6617556822
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 6617556830
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 6617556822
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 6617556828
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 6617556820
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 6617556834
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 6617556818
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 6617556832
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 6617556816
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 6617556842
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 6617556810
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 6617556840
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 6617556808
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 6617556858
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 6617556794
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 6617556856
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 6617556792
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 6617556890
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 6617556762
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 6617556888
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 6617556760
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 6617556954
第 24 筆算術錯誤: 129
第 25 筆錯誤資料: 6617556698
第 25 筆算術錯誤: -127
第 26 筆錯誤資料: 6617556952
第 26 筆

第 334 筆錯誤資料: 6625945429
第 334 筆算術錯誤: 8388604
第 335 筆錯誤資料: 6609168213
第 335 筆算術錯誤: -8388612
第 336 筆錯誤資料: 6634334045
第 336 筆算術錯誤: 16777220
第 337 筆錯誤資料: 6600779613
第 337 筆算術錯誤: -16777212
第 338 筆錯誤資料: 6634334037
第 338 筆算術錯誤: 16777212
第 339 筆錯誤資料: 6600779605
第 339 筆算術錯誤: -16777220
第 340 筆錯誤資料: 6651111261
第 340 筆算術錯誤: 33554436
第 341 筆錯誤資料: 6584002397
第 341 筆算術錯誤: -33554428
第 342 筆錯誤資料: 6651111253
第 342 筆算術錯誤: 33554428
第 343 筆錯誤資料: 6584002389
第 343 筆算術錯誤: -33554436
第 344 筆錯誤資料: 6684665693
第 344 筆算術錯誤: 67108868
第 345 筆錯誤資料: 6550447965
第 345 筆算術錯誤: -67108860
第 346 筆錯誤資料: 6684665685
第 346 筆算術錯誤: 67108860
第 347 筆錯誤資料: 6550447957
第 347 筆算術錯誤: -67108868
第 348 筆錯誤資料: 6751774557
第 348 筆算術錯誤: 134217732
第 349 筆錯誤資料: 6483339101
第 349 筆算術錯誤: -134217724
第 350 筆錯誤資料: 6751774549
第 350 筆算術錯誤: 134217724
第 351 筆錯誤資料: 6483339093
第 351 筆算術錯誤: -134217732
第 352 筆錯誤資料: 6885992285
第 352 筆算術錯誤: 268435460
第 353 筆錯誤資料: 6349121373
第 353 筆算術錯誤: -268435452
第 354 筆錯誤資料: 6885992277
第 354 筆算術錯誤: 268435452
第 355 筆錯誤資料: 634912

第 523 筆錯誤資料: 6617548617
第 523 筆算術錯誤: -8208
第 524 筆錯誤資料: 6617573225
第 524 筆算術錯誤: 16400
第 525 筆錯誤資料: 6617540457
第 525 筆算術錯誤: -16368
第 526 筆錯誤資料: 6617573193
第 526 筆算術錯誤: 16368
第 527 筆錯誤資料: 6617540425
第 527 筆算術錯誤: -16400
第 528 筆錯誤資料: 6617589609
第 528 筆算術錯誤: 32784
第 529 筆錯誤資料: 6617524073
第 529 筆算術錯誤: -32752
第 530 筆錯誤資料: 6617589577
第 530 筆算術錯誤: 32752
第 531 筆錯誤資料: 6617524041
第 531 筆算術錯誤: -32784
第 532 筆錯誤資料: 6617622377
第 532 筆算術錯誤: 65552
第 533 筆錯誤資料: 6617491305
第 533 筆算術錯誤: -65520
第 534 筆錯誤資料: 6617622345
第 534 筆算術錯誤: 65520
第 535 筆錯誤資料: 6617491273
第 535 筆算術錯誤: -65552
第 536 筆錯誤資料: 6617687913
第 536 筆算術錯誤: 131088
第 537 筆錯誤資料: 6617425769
第 537 筆算術錯誤: -131056
第 538 筆錯誤資料: 6617687881
第 538 筆算術錯誤: 131056
第 539 筆錯誤資料: 6617425737
第 539 筆算術錯誤: -131088
第 540 筆錯誤資料: 6617818985
第 540 筆算術錯誤: 262160
第 541 筆錯誤資料: 6617294697
第 541 筆算術錯誤: -262128
第 542 筆錯誤資料: 6617818953
第 542 筆算術錯誤: 262128
第 543 筆錯誤資料: 6617294665
第 543 筆算術錯誤: -262160
第 544 筆錯誤資料: 6618081129
第 544 筆算術錯誤: 524304
第 545 筆錯誤資料: 6617032553
第 545 筆算術錯誤

第 772 筆算術錯誤: 8388672
第 773 筆錯誤資料: 6609168281
第 773 筆算術錯誤: -8388544
第 774 筆錯誤資料: 6625945369
第 774 筆算術錯誤: 8388544
第 775 筆錯誤資料: 6609168153
第 775 筆算術錯誤: -8388672
第 776 筆錯誤資料: 6634334105
第 776 筆算術錯誤: 16777280
第 777 筆錯誤資料: 6600779673
第 777 筆算術錯誤: -16777152
第 778 筆錯誤資料: 6634333977
第 778 筆算術錯誤: 16777152
第 779 筆錯誤資料: 6600779545
第 779 筆算術錯誤: -16777280
第 780 筆錯誤資料: 6651111321
第 780 筆算術錯誤: 33554496
第 781 筆錯誤資料: 6584002457
第 781 筆算術錯誤: -33554368
第 782 筆錯誤資料: 6651111193
第 782 筆算術錯誤: 33554368
第 783 筆錯誤資料: 6584002329
第 783 筆算術錯誤: -33554496
第 784 筆錯誤資料: 6684665753
第 784 筆算術錯誤: 67108928
第 785 筆錯誤資料: 6550448025
第 785 筆算術錯誤: -67108800
第 786 筆錯誤資料: 6684665625
第 786 筆算術錯誤: 67108800
第 787 筆錯誤資料: 6550447897
第 787 筆算術錯誤: -67108928
第 788 筆錯誤資料: 6751774617
第 788 筆算術錯誤: 134217792
第 789 筆錯誤資料: 6483339161
第 789 筆算術錯誤: -134217664
第 790 筆錯誤資料: 6751774489
第 790 筆算術錯誤: 134217664
第 791 筆錯誤資料: 6483339033
第 791 筆算術錯誤: -134217792
第 792 筆錯誤資料: 6885992345
第 792 筆算術錯誤: 268435520
第 793 筆錯誤資料: 6349121433
第 793 筆算術錯誤: -268435392

第 1022 筆算術錯誤: 7680
第 1023 筆錯誤資料: 6617548121
第 1023 筆算術錯誤: -8704
第 1024 筆錯誤資料: 6617573721
第 1024 筆算術錯誤: 16896
第 1025 筆錯誤資料: 6617540953
第 1025 筆算術錯誤: -15872
第 1026 筆錯誤資料: 6617572697
第 1026 筆算術錯誤: 15872
第 1027 筆錯誤資料: 6617539929
第 1027 筆算術錯誤: -16896
第 1028 筆錯誤資料: 6617590105
第 1028 筆算術錯誤: 33280
第 1029 筆錯誤資料: 6617524569
第 1029 筆算術錯誤: -32256
第 1030 筆錯誤資料: 6617589081
第 1030 筆算術錯誤: 32256
第 1031 筆錯誤資料: 6617523545
第 1031 筆算術錯誤: -33280
第 1032 筆錯誤資料: 6617622873
第 1032 筆算術錯誤: 66048
第 1033 筆錯誤資料: 6617491801
第 1033 筆算術錯誤: -65024
第 1034 筆錯誤資料: 6617621849
第 1034 筆算術錯誤: 65024
第 1035 筆錯誤資料: 6617490777
第 1035 筆算術錯誤: -66048
第 1036 筆錯誤資料: 6617688409
第 1036 筆算術錯誤: 131584
第 1037 筆錯誤資料: 6617426265
第 1037 筆算術錯誤: -130560
第 1038 筆錯誤資料: 6617687385
第 1038 筆算術錯誤: 130560
第 1039 筆錯誤資料: 6617425241
第 1039 筆算術錯誤: -131584
第 1040 筆錯誤資料: 6617819481
第 1040 筆算術錯誤: 262656
第 1041 筆錯誤資料: 6617295193
第 1041 筆算術錯誤: -261632
第 1042 筆錯誤資料: 6617818457
第 1042 筆算術錯誤: 261632
第 1043 筆錯誤資料: 6617294169
第 1043 筆算術錯誤: -262656
第 1044 筆錯誤資料: 6618

第 1272 筆錯誤資料: 6617569113
第 1272 筆算術錯誤: 12288
第 1273 筆錯誤資料: 6617552729
第 1273 筆算術錯誤: -4096
第 1274 筆錯誤資料: 6617560921
第 1274 筆算術錯誤: 4096
第 1275 筆錯誤資料: 6617544537
第 1275 筆算術錯誤: -12288
第 1276 筆錯誤資料: 6617577305
第 1276 筆算術錯誤: 20480
第 1277 筆錯誤資料: 6617544537
第 1277 筆算術錯誤: -12288
第 1278 筆錯誤資料: 6617569113
第 1278 筆算術錯誤: 12288
第 1279 筆錯誤資料: 6617536345
第 1279 筆算術錯誤: -20480
第 1280 筆錯誤資料: 6617593689
第 1280 筆算術錯誤: 36864
第 1281 筆錯誤資料: 6617528153
第 1281 筆算術錯誤: -28672
第 1282 筆錯誤資料: 6617585497
第 1282 筆算術錯誤: 28672
第 1283 筆錯誤資料: 6617519961
第 1283 筆算術錯誤: -36864
第 1284 筆錯誤資料: 6617626457
第 1284 筆算術錯誤: 69632
第 1285 筆錯誤資料: 6617495385
第 1285 筆算術錯誤: -61440
第 1286 筆錯誤資料: 6617618265
第 1286 筆算術錯誤: 61440
第 1287 筆錯誤資料: 6617487193
第 1287 筆算術錯誤: -69632
第 1288 筆錯誤資料: 6617691993
第 1288 筆算術錯誤: 135168
第 1289 筆錯誤資料: 6617429849
第 1289 筆算術錯誤: -126976
第 1290 筆錯誤資料: 6617683801
第 1290 筆算術錯誤: 126976
第 1291 筆錯誤資料: 6617421657
第 1291 筆算術錯誤: -135168
第 1292 筆錯誤資料: 6617823065
第 1292 筆算術錯誤: 266240
第 1293 筆錯誤資料: 6617298777
第 1293 筆算術錯誤: -25

第 1522 筆錯誤資料: 6619621209
第 1522 筆算術錯誤: 2064384
第 1523 筆錯誤資料: 6615426905
第 1523 筆算術錯誤: -2129920
第 1524 筆錯誤資料: 6621783897
第 1524 筆算術錯誤: 4227072
第 1525 筆錯誤資料: 6613395289
第 1525 筆算術錯誤: -4161536
第 1526 筆錯誤資料: 6621718361
第 1526 筆算術錯誤: 4161536
第 1527 筆錯誤資料: 6613329753
第 1527 筆算術錯誤: -4227072
第 1528 筆錯誤資料: 6625978201
第 1528 筆算術錯誤: 8421376
第 1529 筆錯誤資料: 6609200985
第 1529 筆算術錯誤: -8355840
第 1530 筆錯誤資料: 6625912665
第 1530 筆算術錯誤: 8355840
第 1531 筆錯誤資料: 6609135449
第 1531 筆算術錯誤: -8421376
第 1532 筆錯誤資料: 6634366809
第 1532 筆算術錯誤: 16809984
第 1533 筆錯誤資料: 6600812377
第 1533 筆算術錯誤: -16744448
第 1534 筆錯誤資料: 6634301273
第 1534 筆算術錯誤: 16744448
第 1535 筆錯誤資料: 6600746841
第 1535 筆算術錯誤: -16809984
第 1536 筆錯誤資料: 6651144025
第 1536 筆算術錯誤: 33587200
第 1537 筆錯誤資料: 6584035161
第 1537 筆算術錯誤: -33521664
第 1538 筆錯誤資料: 6651078489
第 1538 筆算術錯誤: 33521664
第 1539 筆錯誤資料: 6583969625
第 1539 筆算術錯誤: -33587200
第 1540 筆錯誤資料: 6684698457
第 1540 筆算術錯誤: 67141632
第 1541 筆錯誤資料: 6550480729
第 1541 筆算術錯誤: -67076096
第 1542 筆錯誤資料: 6684632921
第 1542 筆算術錯誤: 6

第 1771 筆算術錯誤: -34078720
第 1772 筆錯誤資料: 6685189977
第 1772 筆算術錯誤: 67633152
第 1773 筆錯誤資料: 6550972249
第 1773 筆算術錯誤: -66584576
第 1774 筆錯誤資料: 6684141401
第 1774 筆算術錯誤: 66584576
第 1775 筆錯誤資料: 6549923673
第 1775 筆算術錯誤: -67633152
第 1776 筆錯誤資料: 6752298841
第 1776 筆算術錯誤: 134742016
第 1777 筆錯誤資料: 6483863385
第 1777 筆算術錯誤: -133693440
第 1778 筆錯誤資料: 6751250265
第 1778 筆算術錯誤: 133693440
第 1779 筆錯誤資料: 6482814809
第 1779 筆算術錯誤: -134742016
第 1780 筆錯誤資料: 6886516569
第 1780 筆算術錯誤: 268959744
第 1781 筆錯誤資料: 6349645657
第 1781 筆算術錯誤: -267911168
第 1782 筆錯誤資料: 6885467993
第 1782 筆算術錯誤: 267911168
第 1783 筆錯誤資料: 6348597081
第 1783 筆算術錯誤: -268959744
第 1784 筆錯誤資料: 7154952025
第 1784 筆算術錯誤: 537395200
第 1785 筆錯誤資料: 6081210201
第 1785 筆算術錯誤: -536346624
第 1786 筆錯誤資料: 7153903449
第 1786 筆算術錯誤: 536346624
第 1787 筆錯誤資料: 6080161625
第 1787 筆算術錯誤: -537395200
第 1788 筆錯誤資料: 7691822937
第 1788 筆算術錯誤: 1074266112
第 1789 筆錯誤資料: 5544339289
第 1789 筆算術錯誤: -1073217536
第 1790 筆錯誤資料: 7690774361
第 1790 筆算術錯誤: 1073217536
第 1791 筆錯誤資料: 5543290713
第 1791 筆算術錯誤

第 2021 筆算術錯誤: -2113929216
第 2022 筆錯誤資料: 8731486041
第 2022 筆算術錯誤: 2113929216
第 2023 筆錯誤資料: 4436518745
第 2023 筆算術錯誤: -2181038080
第 2024 筆錯誤資料: 10946078553
第 2024 筆算術錯誤: 4328521728
第 2025 筆錯誤資料: 2356143961
第 2025 筆算術錯誤: -4261412864
第 2026 筆錯誤資料: 10878969689
第 2026 筆算術錯誤: 4261412864
第 2027 筆錯誤資料: 2289035097
第 2027 筆算術錯誤: -4328521728
第 2028 筆錯誤資料: 6818883417
第 2028 筆算術錯誤: 201326592
第 2029 筆錯誤資料: 6550447961
第 2029 筆算術錯誤: -67108864
第 2030 筆錯誤資料: 6684665689
第 2030 筆算術錯誤: 67108864
第 2031 筆錯誤資料: 6416230233
第 2031 筆算術錯誤: -201326592
第 2032 筆錯誤資料: 6953101145
第 2032 筆算術錯誤: 335544320
第 2033 筆錯誤資料: 6416230233
第 2033 筆算術錯誤: -201326592
第 2034 筆錯誤資料: 6818883417
第 2034 筆算術錯誤: 201326592
第 2035 筆錯誤資料: 6282012505
第 2035 筆算術錯誤: -335544320
第 2036 筆錯誤資料: 7221536601
第 2036 筆算術錯誤: 603979776
第 2037 筆錯誤資料: 6147794777
第 2037 筆算術錯誤: -469762048
第 2038 筆錯誤資料: 7087318873
第 2038 筆算術錯誤: 469762048
第 2039 筆錯誤資料: 6013577049
第 2039 筆算術錯誤: -603979776
第 2040 筆錯誤資料: 7758407513
第 2040 筆算術錯誤: 1140850688
第 2041 筆錯誤資料: 5610923865
第 

In [22]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 6310, 1, 6308, 5, 6308, 3, 6306, 9, 6304, 7, 6302, 17, 6296, 15, 6294, 33, 6280, 31, 6278, 65, 6248, 63, 6246, 129, 6184, 127, 6182, 257, 6056, 255, 6054, 513, 5800, 511, 5798, 1025, 5288, 1023, 5286, 2049, 4264, 2047, 4262, 4097, 2216, 4095, 2214, 1882, 4431, 1880, 4429, 3763, 2550, 3761, 2548, 1214, 5099, 1212, 5097, 2427, 3886, 2425, 3884, 4853, 1460, 4851, 1458, 3394, 2919, 3392, 2917, 476, 5837, 474, 5835, 951, 5362, 949, 5360, 1901, 4412, 1899, 4410, 3801, 2512, 3799, 2510, 1290, 5023, 1288, 5021, 2579, 3734, 2577, 3732, 5157, 1156, 5155, 1154, 4002, 2311, 4000, 2309, 1692, 4621, 1690, 4619, 3383, 2930, 3381, 2928, 454, 5859, 452, 5857, 907, 5406, 905, 5404, 1813, 4500, 1811, 4498, 3625, 2688, 3623, 2686, 6, 6309, 2, 6305, 10, 6305, 6, 6301, 18, 6297, 14, 6293, 34, 6281, 30, 6277, 66, 6249, 62, 6245, 130, 6185, 126, 6181, 258, 6057, 254, 6053, 514, 5801, 510, 5797, 1026, 5289, 1022, 5285, 2050, 4265, 2046, 4261, 4098, 2217, 4094, 2213, 1883, 4432, 1879, 4428, 3764,

In [23]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已未去除重複餘數的數量:",np.size(new_R_total))
print("已未去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)

還未去除重複error data的數量: 2178
還未去除重複餘數的數量: 2178
已去除重複error data的數量: 2052
已未去除重複餘數的數量: 2052
已未去除重複AWE的數量: 2052
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 8193, -8191, 8191, -8193, 16385, -16383, 16383, -16385, 32769, -32767, 32767, -32769, 65537, -65535, 65535, -65537, 131073, -131071, 13107

### 所有AWE數量: $4*C^{bits}_{2}$ $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/05/27 使用 Vivado HLX IP 驗證
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [25]:
import math
import time
### 給定一個DEC_R, 找錯誤位置h1及h2(包含錯誤正負號)
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]

start_time = time.time()  # 記錄開始時間
for R, error in zip(new_R_total, new_e_total): 
    found = False                                       # 旗標，用來標示是否要跳出 for H 循環 
    for H in range(0,bits1):                              
        for s in range(-1,2,2):                       
            h1 = s*(H+1)
            ip.write_mm(0, h1)
            R1 = ip.read_mm(4)
            R2 = R - R1
            if R2 < 0:
                R2 = AN + R2
            
            ip.write_mm(8, R2)
            h2 = twos_complement_32bit(ip.read_mm(12))                  
            if (h2 != 0):
                s2 = get_sign(h2)                     
                possible_h1.append(h1)                  # 若找不到對應的h2,h2定為0(函數內容已定義)
                possible_h2.append(h2)                  # 代表錯誤可能是三個或以上(multiple errors)
                output.append((error - (s*2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)
                found = True
                break
        if found:
            break
            
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
print(f"執行時間: {elapsed_time:.6f} 秒")
print("更正後的值是否和原data相同:", data1 == output.pop())
print(elapsed_time / np.size(possible_h1))
##print(possible_h1)
##print(possible_h2)
##print("更正後的值:",output)
print(np.size(possible_h1))
print(np.size(possible_h2))
##print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
##print(possible_SE_h1)

執行時間: 5.010606 秒
更正後的值是否和原data相同: True
0.0024418159302679885
2052
2052
